In [134]:
import json
file = open("data.csv","r")

dataset = file.readlines()
datalist = []
for row in dataset:
    datalist.append(row[:-1] if row[-1] == '\n' else row)
data = []
header = datalist[0].split(',')
for row in datalist[1:]:
    transactionData = {}
    for col in range(len(header)):
        transactionData[header[col]] = row.split(',')[col]
    data.append(transactionData)


In [130]:
from hashlib import sha256
import json
import time

from flask import Flask, request
import requests


class Block:
    def __init__(self, index, transactions, timestamp, previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = 0

    def compute_hash(self):
        """
        A function that return the hash of the block contents.
        """
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_string.encode()).hexdigest()



In [131]:

class Blockchain:
    # difficulty of our PoW algorithm
    difficulty = 2

    def __init__(self):
        self.unconfirmed_transactions = []
        self.chain = []
        self.create_genesis_block()

    def create_genesis_block(self):
        """
        A function to generate genesis block and appends it to
        the chain. The block has index 0, previous_hash as 0, and
        a valid hash.
        """
        genesis_block = Block(0, [], time.time(), "0")
        genesis_block.hash = genesis_block.compute_hash()
        self.chain.append(genesis_block)

    @property
    def last_block(self):
        return self.chain[-1]

    def add_block(self, block, proof):
        """
        A function that adds the block to the chain after verification.
        Verification includes:
        * Checking if the proof is valid.
        * The previous_hash referred in the block and the hash of latest block
          in the chain match.
        """
        previous_hash = self.last_block.hash

        if previous_hash != block.previous_hash:
            return False

        if not self.is_valid_proof(block, proof):
            return False

        block.hash = proof
        self.chain.append(block)
        return True

    def is_valid_proof(self, block, block_hash):
        """
        Check if block_hash is valid hash of block and satisfies
        the difficulty criteria.
        """
        return (block_hash.startswith('0' * Blockchain.difficulty) and
                block_hash == block.compute_hash())

    def proof_of_work(self, block):
        """
        Function that tries different values of nonce to get a hash
        that satisfies our difficulty criteria.
        """
        block.nonce = 0

        computed_hash = block.compute_hash()
        while not computed_hash.startswith('0' * Blockchain.difficulty):
            block.nonce += 1
            computed_hash = block.compute_hash()

        return computed_hash

    def add_new_transaction(self, transaction):
        self.unconfirmed_transactions.append(transaction)

    def mine(self):
        """
        This function serves as an interface to add the pending
        transactions to the blockchain by adding them to the block
        and figuring out Proof Of Work.
        """
        if not self.unconfirmed_transactions:
            return False

        last_block = self.last_block

        new_block = Block(index=last_block.index + 1,
                          transactions=self.unconfirmed_transactions,
                          timestamp=time.time(),
                          previous_hash=last_block.hash)

        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)

        self.unconfirmed_transactions = []
        return new_block.index


In [145]:
blockchain = Blockchain()
for transaction in data:
    blockchain.add_new_transaction(transaction)
    blockchain.mine()


In [149]:
def get_chain():
    chain_data = []
    for block in blockchain.chain:
        chain_data.append(block.__dict__)
    return json.dumps({"length": len(chain_data),
                       "chain": chain_data},indent=2)
print(get_chain())

{
  "length": 15,
  "chain": [
    {
      "index": 0,
      "transactions": [],
      "timestamp": 1695988468.6782646,
      "previous_hash": "0",
      "nonce": 0,
      "hash": "47875481c50212ec67cf67f2199b3836b2b6bad31c0e93d883737271b1ef2580"
    },
    {
      "index": 1,
      "transactions": [
        {
          "TransactionNo": "581482",
          "Date": "12-09-2019",
          "ProductNo": "22485",
          "ProductName": "Set Of 2 Wooden Market Crates",
          "Price": "21.47",
          "Quantity": "12",
          "CustomerNo": "17490",
          "Country": "United Kingdom"
        }
      ],
      "timestamp": 1695988468.6782646,
      "previous_hash": "47875481c50212ec67cf67f2199b3836b2b6bad31c0e93d883737271b1ef2580",
      "nonce": 627,
      "hash": "00ac08b8cae0a4476dcd7f71e33b5065f58604de1d92abd57317f60af3d41b31"
    },
    {
      "index": 2,
      "transactions": [
        {
          "TransactionNo": "581475",
          "Date": "12-09-2019",
          "Product